In [11]:
class Category:
    def __init__(self, src:dict, target:dict, comp:dict, addIdentities:bool = False) -> None:
        self.src = src
        self.target = target
        self.comp = comp
        self.objects = set(src.values()).union(set(target.values()))
        self.morphisms = list(src.keys())

        assert src.keys() == target.keys()
        assert self.composabilityCheck()
        assert self.compositionCheck()
        assert self.associativityCheck()

        if addIdentities:
            self.addIdentities()
        
    def __str__(self) -> str:
        return str((self.src, self.target, self.comp))

    def composabilityCheck(self)-> bool:
        return True
    
    def compositionCheck(self) -> bool:
        return True
    
    def associativityCheck(self) -> bool:
        return True
    
    def addIdentities(self) -> None:
        for thisObject in self.objects:
            thisMorphism = "id_" + str(thisObject)
            self.src[thisMorphism] = thisObject
            self.target[thisMorphism] = thisObject
            self.morphisms = self.morphisms + [thisMorphism]
            for i in self.morphisms:
                if self.src[i] == thisObject:
                    self.comp[(i, thisMorphism)] = i
                if self.target[i] == thisObject:
                    self.comp[(thisMorphism, i)] = i


class Functor:
    def __init__(self, srcCat: Category, targetCat: Category, objectMap: dict, morphismMap: dict) -> None:
        self.srcCat = srcCat
        self.targetCat = targetCat
        self.objectMap = objectMap
        self.morphismMap = morphismMap

        assert self.homomorphismFunctorCheck()

    def homomorphismFunctorCheck(self) -> bool:
        return True

In [35]:
def productCategory(c:Category, d:Category):
    src = srcProduct(c, d)
    target = targetProduct(c, d)
    comp = compProduct(c,d)
    productCat = Category(src, target, comp, addIdentities=False)
    return productCat

def srcProduct(c:Category, d:Category):
    src = {(m1,m2):(o1, o2) for m1, o1 in c.src.items() for m2,o2 in d.src.items()}
    return src

def targetProduct(c:Category, d:Category):
    target = {(m1,m2):(o1, o2) for m1, o1 in c.target.items() for m2,o2 in d.target.items()}
    return target
    
def compProduct(c:Category, d:Category):
    comp = {((m1Source[0],m2Source[0]), (m1Source[1],m2Source[1])):(m1Target, m2Target) for m1Source, m1Target in c.comp.items() for m2Source, m2Target in d.comp.items()}
    return comp

In [36]:
src = {"f":0, "g":1, "h":2, "i":0, "m":0, "k":1}
target = {"f":1, "g":2, "h":3, "i":2, "m":3, "k":3}
comp = {("g","f"): "i", ("h", "g"):"k", ("h","i"): "m", ("k","f"): "m"}

c1 = Category(src, target, comp, True)
c2 = Category(src, target, comp, True)

productC1C2 = productCategory(c1, c2)

In [37]:
productC1C2.comp

{(('g', 'g'), ('f', 'f')): ('i', 'i'),
 (('g', 'h'), ('f', 'g')): ('i', 'k'),
 (('g', 'h'), ('f', 'i')): ('i', 'm'),
 (('g', 'k'), ('f', 'f')): ('i', 'm'),
 (('g', 'f'), ('f', 'id_0')): ('i', 'f'),
 (('g', 'i'), ('f', 'id_0')): ('i', 'i'),
 (('g', 'm'), ('f', 'id_0')): ('i', 'm'),
 (('g', 'id_0'), ('f', 'id_0')): ('i', 'id_0'),
 (('g', 'id_1'), ('f', 'f')): ('i', 'f'),
 (('g', 'g'), ('f', 'id_1')): ('i', 'g'),
 (('g', 'k'), ('f', 'id_1')): ('i', 'k'),
 (('g', 'id_1'), ('f', 'id_1')): ('i', 'id_1'),
 (('g', 'id_2'), ('f', 'g')): ('i', 'g'),
 (('g', 'h'), ('f', 'id_2')): ('i', 'h'),
 (('g', 'id_2'), ('f', 'i')): ('i', 'i'),
 (('g', 'id_2'), ('f', 'id_2')): ('i', 'id_2'),
 (('g', 'id_3'), ('f', 'h')): ('i', 'h'),
 (('g', 'id_3'), ('f', 'm')): ('i', 'm'),
 (('g', 'id_3'), ('f', 'k')): ('i', 'k'),
 (('g', 'id_3'), ('f', 'id_3')): ('i', 'id_3'),
 (('h', 'g'), ('g', 'f')): ('k', 'i'),
 (('h', 'h'), ('g', 'g')): ('k', 'k'),
 (('h', 'h'), ('g', 'i')): ('k', 'm'),
 (('h', 'k'), ('g', 'f')): ('k'

In [ ]:
#Check that all the composables defined by equations have matching targets and sources.
function checkComposability(c::Category, debug::Bool=false)

    for thisEquality in c.relations
        if !occursin("~", string(thisEquality.left))
            second, first = thisEquality.left.args 
            second = string(second)
            first = string(first)
            
            if c.src[second] != c.target[first]
                if debug
                    println("composability violation")
                    println(first,second)
                    println(" ")
                end
                return false
            end
        end
    end
    return true
end

#Check that the composition of any two arrows with matching sources and targets has a name by brute force
function checkComposition(c::Category, debug::Bool=false) :: Bool
    
    morphisms = [morph for morph in keys(c.src) if !occursin("id", morph)]
    for i in morphisms
        for j in morphisms
            if c.target[i] == c.src[j]
                simplified = simplifyComposition(i, j, c.relations, c.identityRelations)

                if occursin("comp", simplified)
                    if debug
                        println("missing composition name: ")
                        println(thisComposition)
                        println(" ")
                    end
                    return false
                end
            end
        end
    end
    return true
end;

# Check that category c satisfies associativity by brute force
function checkAssociativity(c::Category, debug::Bool = false) :: Bool
    morphisms = keys(c.src)
    relations = c.relations
    identityRelations = c.identityRelations

    println("Checking associativity...")
    for i in morphisms
        for j in morphisms
            for k in morphisms
                if (c.target[i] == c.src[j]) && (c.target[j] == c.src[k])
                    if occursin("id", i * j * k)
                        continue
                    end
                    leftAssocLeft = simplifyComposition(i, j, relations, identityRelations)
                    leftAssoc = simplifyComposition(leftAssocLeft, k, relations, identityRelations)
                    rightAssocRight = simplifyComposition(j, k, relations, identityRelations)
                    rightAssoc = simplifyComposition(i, rightAssocRight, relations, identityRelations)
                    if leftAssoc != rightAssoc
                        if debug
                            print("associativity violation: ")
                            println(k,j,i)
                            println(leftAssocLeft)
                            println(leftAssoc)
                            println(rightAssocRight)
                            println(rightAssoc)
                            println(" ")
                        end

                        return false
                    end
                end
            end
        end
    end

    return true
end;



function homomorphismFunctorCheck(functor)
    srcCat = functor.srcCat
    targetCat = functor.targetCat

    for i in srcCat.morphisms
        src = srcCat.src[i]
        target = srcCat.target[i]

        # check source and target of morphism image
        if (targetCat.src[functor.morphismMap[i]] != functor.objectMap[src]) || (targetCat.target[functor.morphismMap[i]] != functor.objectMap[target])
            println("Functor violation...")
            return false
        end

        # check homomorphism
        for j in srcCat.morphisms
            if srcCat.target[i] == srcCat.src[j]

            end
        end
    end
    return true
end

